In [4]:
import os
import datetime

In [5]:
class Downloader:
    def __init__(self):
        self.tar_path = '/home/ubuntu/synthetic/tar'
        self.bin_path = '/mnt/hgfs/kagoshima/bin'  # Seminarをマウントしている
        %mkdir -p $self.tar_path
        #self.delete_directory(tar_flag=True)
        %mkdir -p $self.tar_path
        %mkdir -p $self.bin_path
        %cd ~/synthetic/tar
    
    def set_date(self, dt):
        # URLを指定
        directory = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original'
        date = dt.strftime('%Y/%m/%d')
        self.timestamp = dt.strftime('%Y%m%d%H%M%S')
        self.tar_filename = 'Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV__grib2.tar'
        self.URL = directory +'/'+ date +'/'+ self.tar_filename
        self.bin_path_year = self.bin_path + '/' + dt.strftime('%Y')
        %mkdir -p $self.bin_path_year
        
    def get_bin_file(self):
        # binファイルがあるかを確認
        bin_file_path = self.bin_path_year+'/Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin'
        if os.path.exists(bin_file_path):
            #print('already exist: ',bin_file_path)
            self.out_available(1)
            #pass
        else:
            # tarファイルを~/tarに保存
            wget_result = !wget -nc -P $self.tar_path $self.URL
            download_error = [s for s in wget_result if 'エラー' in s]
            if download_error:  # ダウンロードエラーの場合
                self.out_error_URL('download error')
            else:
                # tarファイルから1kmメッシュデータのみを取り出す
                filelist = !tar -tf $self.tar_filename  # tarファイル内ファイル名リスト
                ggis_name = [s for s in filelist if 'Ggis1km' in s]  # ファイル名を取得
                if ggis_name:
                    ggis_name = ggis_name[0]
                    !tar -C $self.bin_path_year -xvf $self.tar_filename $ggis_name # tarファイルから1kmメッシュ全国合成レーダーエコー強度GPVのみを/binに取り出す
                    !rm $self.tar_filename  # tarファイルは不要なので削除
                    self.out_available(1)
                else:
                    self.out_error_URL('tarfile error')
                        
    def out_error_URL(self, error_name):
        print(error_name+': '+self.URL)
        with open('/mnt/hgfs/kagoshima/bin/error_URL_'+self.timestamp[:4]+'.csv', 'a') as f:
            f.write(error_name+','+self.URL+'\n')
        self.out_available(0)
    
    def out_available(self, n):
        with open('/mnt/hgfs/kagoshima/bin/available.csv', 'a') as f:
            f.write(self.timestamp+','+str(n)+'\n')
    
    def delete_directory(self, tar_flag=False, bin_flag=False):
        if tar_flag:
            !rm -r $self.tar_path
        if bin_flag:
            !rm -r $self.bin_path

In [ ]:
start = '2005/10/21 00:00:00'
end = '2019/01/01 00:00:00'
dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')

downloader = Downloader()
while dt < dt_limit:
    downloader.set_date(dt)
    downloader.get_bin_file()
    dt = dt + datetime.timedelta(minutes=10)

/home/ubuntu/synthetic/tar
download error: http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original/2005/10/22/Z__C_RJTD_20051022012000_RDR_JMAGPV__grib2.tar
download error: http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original/2005/10/24/Z__C_RJTD_20051024031000_RDR_JMAGPV__grib2.tar
download error: http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original/2005/10/28/Z__C_RJTD_20051028070000_RDR_JMAGPV__grib2.tar
download error: http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original/2005/10/28/Z__C_RJTD_20051028071000_RDR_JMAGPV__grib2.tar
